### Deep Sentiment Analysis Tutorial

### Setup GPU
- On Google Colab make sure you select Python 3/GPU runtime before running the code

In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


### Download Data

In [2]:
![ ! -d data ] && mkdir data/
![ -f data/aclImdb_v1.tar.gz ] && echo "Skip Download"
![ ! -f data/aclImdb_v1.tar.gz ] && wget -N https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -P data/

Skip Download


In [3]:
%%time
![ -d data/aclImdb/ ] && echo "Data already extracted"
![ ! -d data/aclImdb/ ] && tar -xzf data/aclImdb_v1.tar.gz -C data/

Data already extracted
CPU times: user 6.83 ms, sys: 6.68 ms, total: 13.5 ms
Wall time: 224 ms


### Imports

In [4]:
import os
import re
import nltk
from collections import Counter
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
from tensorflow.contrib import seq2seq
from tensorflow.contrib.rnn import DropoutWrapper
import random

In [5]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/bishal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
MAX_SEQ_LEN = 50
BATCH_SIZE = 64

In [8]:
class Lang:
    def __init__(self, counter, vocab_size):
        self.word2id = {}
        self.id2word = {}
        self.pad = "<PAD>"
        self.sos = "<SOS>"
        self.eos = "<EOS>"
        self.unk = "<UNK>"
        
        self.ipad = 0
        self.isos = 1
        self.ieos = 2
        self.iunk = 3
        
        self.word2id[self.pad] = 0
        self.word2id[self.sos] = 1
        self.word2id[self.eos] = 2
        self.word2id[self.unk] = 3
        
        self.id2word[0] = self.pad
        self.id2word[1] = self.sos
        self.id2word[2] = self.eos
        self.id2word[3] = self.unk
        
        curr_id = 4
        for w, c in counter.most_common(vocab_size):
            self.word2id[w] = curr_id
            self.id2word[curr_id] = w
            curr_id += 1
    
    def encodeSentence(self, wseq, max_len=-1):
        # wseq = nltk.tokenize.word_tokenize(s.lower().strip())
        if max_len == -1:
            return [self.word2id[w] if w in self.word2id else self.iunk for w in wseq]
        else:
            return ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + [self.ieos] + [self.ipad]*max_len)[:max_len]
        
    def encodeSentence2(self, wseq, max_len=-1):
        # wseq = nltk.tokenize.word_tokenize(s.lower().strip()) 
        return min(max_len, len(wseq)+1), \
            ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + \
                [self.ieos] + [self.ipad]*max_len)[:max_len]
    
    def decodeSentence(self, id_seq):
        id_seq = np.array(id_seq + [self.ieos])
        j = np.argmax(id_seq==self.ieos)
        s = ' '.join([self.id2word[x] for x in id_seq[:j]])
        s = s.replace(self.unk, "UNK")
        return s

### Let's read in the data

In [9]:
data_folder = 'data/aclImdb/'

In [10]:
rp = os.path.join(data_folder, 'train/pos')
train_positive = [os.path.join(rp, f) for f in os.listdir(rp)]
rp = os.path.join(data_folder, 'train/neg')
train_negative = [os.path.join(rp, f) for f in os.listdir(rp)]

rp = os.path.join(data_folder, 'test/pos')
test_positive = [os.path.join(rp, f) for f in os.listdir(rp)]
rp = os.path.join(data_folder, 'test/neg')
test_negative = [os.path.join(rp, f) for f in os.listdir(rp)]

#### Limit number of samples
To quickly train a small model, consider setting n_train and n_test to some relatively small numbers e.g. `1000`. Set, 
`n_train = n_test = -1` to use all the samples available.

In [11]:
n_train = 100000
n_test = 2500

In [12]:
re_html_cleaner = re.compile(r"<.*?>")

In [13]:
en_counter = Counter()
train_data = []
for _fname in tqdm_notebook(train_positive[:n_train], desc="Crunching +ve samples: "):
    with open(_fname) as f:
        text = f.read().strip()
        text = re_html_cleaner.sub(" ", text)
        wseq = nltk.tokenize.word_tokenize(text.lower())
        en_counter += Counter(wseq)
        train_data.append((wseq, 1))
        
for _fname in tqdm_notebook(train_negative[:n_train], desc="Crunching -ve samples: "):
    with open(_fname) as f:
        text = f.read().strip()
        text = re_html_cleaner.sub(" ", text)
        wseq = nltk.tokenize.word_tokenize(text.lower())
        en_counter += Counter(wseq)
        train_data.append((wseq, 0))

In [14]:
test_data = []
for _fname in tqdm_notebook(test_positive[:n_test], desc="Crunching +ve samples: "):
    with open(_fname) as f:
        text = f.read().strip()
        text = re_html_cleaner.sub(" ", text)
        wseq = nltk.tokenize.word_tokenize(text.lower())
        test_data.append((wseq, 1))
        
for _fname in tqdm_notebook(test_negative[:n_test], desc="Crunching -ve samples: "):
    with open(_fname) as f:
        text = f.read().strip()
        text = re_html_cleaner.sub(" ", text)
        wseq = nltk.tokenize.word_tokenize(text.lower())
        test_data.append((wseq, 0))

In [15]:
# A few sample english words
print("\nMost common en words in dataset:\n", en_counter.most_common(10))

print("\nTotal (en)words gathered from dataset:", len(en_counter))


Most common en words in dataset:
 [('the', 334752), (',', 275881), ('.', 271448), ('and', 163327), ('a', 162162), ('of', 145428), ('to', 135195), ('is', 110396), ('it', 95772), ('in', 93249)]

Total (en)words gathered from dataset: 105920


In [16]:
V = 10000

In [17]:
en_lang = Lang(en_counter, V)

In [18]:
wseq = nltk.tokenize.word_tokenize("Where are you going?".lower())
print("Test en encoding:", en_lang.encodeSentence(wseq))
print("Test en decoding:", en_lang.decodeSentence(en_lang.encodeSentence(wseq, 10)))

Test en encoding: [131, 33, 27, 182, 58]
Test en decoding: where are you going ?


### The RNN based Sentence Classifier architecture
- We will implement a RNN based classifier architecture for sentiment analysis in Tensorflow r1.13.1 / r1.14
- Debugging Tip: Always keep track of tensor dimensions!
- **Tensorflow Computation Graph** - We will build a tf computation graph first. This is the representation used by tf for any neural network architecture. Once the computation graph is built, you can feed data to it for training or inference

#### Word Embedding Matrix

In [19]:
en_word_emb_matrix = tf.get_variable("en_word_emb_matrix", (V, 300), dtype=tf.float32)

Instructions for updating:
Colocations handled automatically by placer.


#### Placeholders

In [20]:
keep_prob = tf.placeholder(tf.float32)

In [21]:
input_ids = tf.placeholder(tf.int32, (None, MAX_SEQ_LEN))
input_lens = tf.placeholder(tf.int32, (None, ))

In [22]:
y_placeholder = tf.placeholder(tf.int32, (None,))

#### Tensorflow Graphs

In [23]:
input_emb = tf.nn.embedding_lookup(en_word_emb_matrix, input_ids)

In [24]:
input_emb.shape

TensorShape([Dimension(None), Dimension(50), Dimension(300)])

#### Encoder

##### RNN Units

In [25]:
encoder_cell = tf.nn.rnn_cell.GRUCell(128)
encoder_cell = DropoutWrapper(encoder_cell, output_keep_prob=keep_prob)

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.


In [26]:
enc_outputs, enc_state = tf.nn.dynamic_rnn(
    encoder_cell, input_emb, sequence_length=input_lens, initial_state=encoder_cell.zero_state(BATCH_SIZE, dtype=tf.float32)
)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [27]:
enc_outputs.shape

TensorShape([Dimension(64), Dimension(50), Dimension(128)])

In [28]:
enc_state.shape

TensorShape([Dimension(64), Dimension(128)])

### Classifier Layer

In [29]:
dense_layer = tf.layers.Dense(1)

#### Approaches:
As input to the final linear layers use mean of the hidden states?

or

As input to the final linear layers use the last hidden state?

##### Approch 1: Take mean of enc_outputs across dimension 1
- **IMPORTANT:** Need to **mask** the positions in input sentence that doesn't contain any inputs

In [30]:
# masks = tf.sequence_mask(input_lens, MAX_SEQ_LEN, dtype=tf.float32, name='masks')
# class_prob = tf.nn.sigmoid(
#                 dense_layer(
#                     tf.reduce_mean(
#                         enc_outputs*masks[:, :, None], 1)
#                 )
# ) 

# print(class_prob.shape)

##### Approch 2: Use enc_state (final hidden state)

In [31]:
class_prob = tf.nn.sigmoid(dense_layer(enc_state))
print(class_prob.shape)

(64, 1)


#### Loss and Optimizers [softmax_cross_entropy]
Note that `onehot_labels` and `logits` must have the same shape, e.g. `[batch_size, num_classes]`

In [32]:
print(y_placeholder.shape)
print(class_prob.shape)

(?,)
(64, 1)


In [33]:
# Loss function - softmax cross entropy
y_ = tf.cast(y_placeholder[:, None], dtype=tf.float32)
cost = -y_*tf.log(class_prob + 1e-12) - (1-y_)*tf.log(1-class_prob + 1e-12)
cost = tf.reduce_mean(cost)

# Optimizer
optimizer = tf.train.AdamOptimizer(0.0001)

In [34]:
train_op = optimizer.minimize(cost)

In [35]:
init = tf.global_variables_initializer()

#### Tensorflow Sessions

In [36]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True

In [37]:
sess = tf.InteractiveSession(config=sess_config)
sess.run(init)

#### Minibatch Training

In [38]:
random.seed(41)

In [39]:
random.shuffle(train_data)

In [40]:
train_n = len(train_data)

In [41]:
test_n = len(test_data)

In [42]:
def small_test():
    all_true = []
    all_preds = []
    for m in range(0, test_n, BATCH_SIZE):
        n = m + BATCH_SIZE
        if n > test_n:
            break

        input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
        input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
        true_class_batch = np.zeros((BATCH_SIZE))
        for i in range(m, n):
            b,a = en_lang.encodeSentence2(test_data[i][0], MAX_SEQ_LEN)
            input_batch[i-m,:] = a
            input_lens_batch[i-m] = b
            true_class_batch[i-m] = test_data[i][1]

        feed_dict={
            input_ids: input_batch,
            input_lens: input_lens_batch,
            keep_prob: 1.0
        }
        pred_batch = sess.run(class_prob, feed_dict=feed_dict)
        # acc = accuracy_score(true_class_batch, pred_batch > 0.5)
        all_true.extend(list(true_class_batch))
        all_preds.extend(list(pred_batch[:,0]))
    
    all_true = np.array(all_true)
    all_preds = np.array(all_preds)
    prec = precision_score(all_true, all_preds > 0.5)*100
    rec = recall_score(all_true, all_preds > 0.5)*100
    f1 = f1_score(all_true, all_preds > 0.5)*100
    print(f"Precision: {prec:2.2F}, Recall: {rec:2.2F}, F1-Score: {f1:2.2F}")

In [43]:
for _e in range(10):
    # Mix things up a bit.
    random.shuffle(train_data)
    pbar = tqdm_notebook(range(0, train_n, BATCH_SIZE))
    batch_loss = 0
    bxi = 0
    for m in pbar:
        n = m + BATCH_SIZE
        if n <= train_n:
            # print("Epoch Complete... \n")

            input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
            input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
            true_class_batch = np.zeros((BATCH_SIZE))
            for i in range(m, n):
                b,a = en_lang.encodeSentence2(train_data[i][0], MAX_SEQ_LEN)
                input_batch[i-m,:] = a
                input_lens_batch[i-m] = b
                true_class_batch[i-m] = train_data[i][1]

            feed_dict={
                input_ids: input_batch,
                input_lens: input_lens_batch,
                y_placeholder: true_class_batch,
                keep_prob: 0.6
            }
            sess.run(train_op, feed_dict=feed_dict)
            batch_loss += sess.run(cost, feed_dict=feed_dict)
            pbar.set_description(f"Epoch: {_e} >> Loss: {batch_loss/(bxi+1):2.2F}:")
            bxi += 1
            if (1 + n//BATCH_SIZE) % 10 == 0:
                small_test()

Precision: 50.74, Recall: 80.44, F1-Score: 62.23
Precision: 50.76, Recall: 92.76, F1-Score: 65.61
Precision: 50.61, Recall: 97.24, F1-Score: 66.58
Precision: 51.75, Recall: 86.72, F1-Score: 64.82
Precision: 52.54, Recall: 84.52, F1-Score: 64.80
Precision: 54.47, Recall: 76.56, F1-Score: 63.65
Precision: 56.35, Recall: 71.20, F1-Score: 62.91
Precision: 57.53, Recall: 60.08, F1-Score: 58.78
Precision: 59.51, Recall: 53.32, F1-Score: 56.24
Precision: 63.29, Recall: 40.76, F1-Score: 49.59
Precision: 63.80, Recall: 49.84, F1-Score: 55.96
Precision: 76.80, Recall: 13.64, F1-Score: 23.17
Precision: 70.77, Recall: 46.20, F1-Score: 55.91
Precision: 62.87, Recall: 74.04, F1-Score: 68.00
Precision: 67.55, Recall: 68.36, F1-Score: 67.95
Precision: 71.00, Recall: 62.76, F1-Score: 66.62
Precision: 67.52, Recall: 74.08, F1-Score: 70.65
Precision: 73.48, Recall: 59.96, F1-Score: 66.04
Precision: 65.91, Recall: 81.52, F1-Score: 72.89
Precision: 72.93, Recall: 64.44, F1-Score: 68.42
Precision: 67.07, Re

Precision: 70.45, Recall: 81.84, F1-Score: 75.72
Precision: 75.92, Recall: 71.88, F1-Score: 73.84
Precision: 75.89, Recall: 71.28, F1-Score: 73.51
Precision: 75.09, Recall: 72.84, F1-Score: 73.95
Precision: 72.47, Recall: 78.24, F1-Score: 75.25
Precision: 75.10, Recall: 72.64, F1-Score: 73.85
Precision: 74.96, Recall: 71.36, F1-Score: 73.11
Precision: 72.42, Recall: 77.52, F1-Score: 74.88
Precision: 74.91, Recall: 73.80, F1-Score: 74.35
Precision: 74.69, Recall: 74.48, F1-Score: 74.58
Precision: 73.85, Recall: 73.20, F1-Score: 73.52
Precision: 74.28, Recall: 74.76, F1-Score: 74.52
Precision: 73.04, Recall: 77.92, F1-Score: 75.40
Precision: 72.79, Recall: 76.60, F1-Score: 74.64
Precision: 76.84, Recall: 69.28, F1-Score: 72.86
Precision: 71.42, Recall: 79.96, F1-Score: 75.45
Precision: 79.64, Recall: 62.28, F1-Score: 69.90
Precision: 73.08, Recall: 77.52, F1-Score: 75.23
Precision: 72.75, Recall: 78.48, F1-Score: 75.51
Precision: 77.33, Recall: 69.16, F1-Score: 73.02
Precision: 76.10, Re

Precision: 73.69, Recall: 76.40, F1-Score: 75.02
Precision: 78.81, Recall: 67.24, F1-Score: 72.57
Precision: 70.64, Recall: 81.24, F1-Score: 75.57
Precision: 77.25, Recall: 70.24, F1-Score: 73.58
Precision: 77.08, Recall: 70.76, F1-Score: 73.79
Precision: 74.09, Recall: 74.24, F1-Score: 74.17
Precision: 78.40, Recall: 68.24, F1-Score: 72.97
Precision: 74.59, Recall: 72.68, F1-Score: 73.62
Precision: 75.25, Recall: 73.96, F1-Score: 74.60
Precision: 71.82, Recall: 77.88, F1-Score: 74.73
Precision: 76.41, Recall: 69.56, F1-Score: 72.82
Precision: 71.31, Recall: 80.12, F1-Score: 75.46
Precision: 73.86, Recall: 74.16, F1-Score: 74.01
Precision: 79.86, Recall: 62.16, F1-Score: 69.91
Precision: 70.70, Recall: 80.96, F1-Score: 75.48
Precision: 77.59, Recall: 66.60, F1-Score: 71.67
Precision: 77.29, Recall: 68.48, F1-Score: 72.62
Precision: 70.30, Recall: 81.44, F1-Score: 75.46
Precision: 77.25, Recall: 67.64, F1-Score: 72.13
Precision: 75.70, Recall: 72.04, F1-Score: 73.83
Precision: 77.93, Re

Precision: 75.24, Recall: 71.96, F1-Score: 73.56
Precision: 73.82, Recall: 74.00, F1-Score: 73.91
Precision: 74.29, Recall: 73.64, F1-Score: 73.97
Precision: 74.35, Recall: 72.80, F1-Score: 73.57
Precision: 73.28, Recall: 73.16, F1-Score: 73.22
Precision: 73.44, Recall: 74.32, F1-Score: 73.88
Precision: 74.15, Recall: 71.72, F1-Score: 72.92
Precision: 75.28, Recall: 70.16, F1-Score: 72.63
Precision: 74.57, Recall: 71.44, F1-Score: 72.97
Precision: 75.39, Recall: 70.08, F1-Score: 72.64
Precision: 72.68, Recall: 74.28, F1-Score: 73.47
Precision: 74.68, Recall: 71.24, F1-Score: 72.92
Precision: 73.32, Recall: 73.20, F1-Score: 73.26
Precision: 74.08, Recall: 71.80, F1-Score: 72.92
Precision: 74.47, Recall: 71.28, F1-Score: 72.84
Precision: 72.85, Recall: 75.44, F1-Score: 74.12
Precision: 75.36, Recall: 70.36, F1-Score: 72.78
Precision: 73.24, Recall: 73.36, F1-Score: 73.30
Precision: 74.25, Recall: 72.32, F1-Score: 73.27
Precision: 74.88, Recall: 70.72, F1-Score: 72.74
Precision: 74.40, Re

Precision: 74.02, Recall: 73.16, F1-Score: 73.59
Precision: 75.87, Recall: 69.04, F1-Score: 72.29
Precision: 73.10, Recall: 75.00, F1-Score: 74.04
Precision: 74.95, Recall: 70.36, F1-Score: 72.58
Precision: 73.87, Recall: 72.24, F1-Score: 73.04
Precision: 74.07, Recall: 72.20, F1-Score: 73.12
Precision: 73.97, Recall: 71.16, F1-Score: 72.54
Precision: 74.29, Recall: 72.36, F1-Score: 73.31
Precision: 73.85, Recall: 72.08, F1-Score: 72.96
Precision: 73.44, Recall: 72.00, F1-Score: 72.71
Precision: 75.43, Recall: 69.88, F1-Score: 72.55
Precision: 72.64, Recall: 74.88, F1-Score: 73.74
Precision: 73.34, Recall: 73.96, F1-Score: 73.65
Precision: 73.70, Recall: 73.08, F1-Score: 73.39
Precision: 71.00, Recall: 77.48, F1-Score: 74.10
Precision: 76.80, Recall: 69.00, F1-Score: 72.69
Precision: 72.80, Recall: 74.96, F1-Score: 73.87
Precision: 72.93, Recall: 75.12, F1-Score: 74.01
Precision: 72.18, Recall: 75.64, F1-Score: 73.87
Precision: 76.48, Recall: 69.96, F1-Score: 73.07
Precision: 74.77, Re

Precision: 73.64, Recall: 73.20, F1-Score: 73.42
Precision: 74.67, Recall: 69.92, F1-Score: 72.22
Precision: 73.96, Recall: 70.56, F1-Score: 72.22
Precision: 73.60, Recall: 72.36, F1-Score: 72.97
Precision: 75.13, Recall: 69.24, F1-Score: 72.06
Precision: 72.66, Recall: 72.40, F1-Score: 72.53
Precision: 74.60, Recall: 70.60, F1-Score: 72.54
Precision: 72.22, Recall: 74.56, F1-Score: 73.37
Precision: 76.74, Recall: 65.44, F1-Score: 70.64
Precision: 71.94, Recall: 75.80, F1-Score: 73.82
Precision: 75.33, Recall: 69.36, F1-Score: 72.22
Precision: 72.63, Recall: 75.88, F1-Score: 74.22
Precision: 75.16, Recall: 68.64, F1-Score: 71.75
Precision: 72.84, Recall: 75.00, F1-Score: 73.91
Precision: 72.89, Recall: 75.08, F1-Score: 73.97
Precision: 75.02, Recall: 69.56, F1-Score: 72.19
Precision: 73.04, Recall: 74.56, F1-Score: 73.79
Precision: 74.28, Recall: 70.00, F1-Score: 72.08
Precision: 73.20, Recall: 73.32, F1-Score: 73.26
Precision: 74.03, Recall: 72.28, F1-Score: 73.14
Precision: 73.24, Re

Precision: 72.06, Recall: 75.84, F1-Score: 73.90
Precision: 74.93, Recall: 69.00, F1-Score: 71.85
Precision: 73.14, Recall: 72.88, F1-Score: 73.01
Precision: 73.81, Recall: 71.60, F1-Score: 72.69
Precision: 74.17, Recall: 71.08, F1-Score: 72.59
Precision: 73.48, Recall: 71.92, F1-Score: 72.69
Precision: 72.20, Recall: 74.60, F1-Score: 73.38
Precision: 74.62, Recall: 70.08, F1-Score: 72.28
Precision: 72.15, Recall: 75.32, F1-Score: 73.70
Precision: 74.73, Recall: 68.60, F1-Score: 71.53
Precision: 73.64, Recall: 70.84, F1-Score: 72.21
Precision: 72.81, Recall: 71.24, F1-Score: 72.02
Precision: 73.65, Recall: 70.56, F1-Score: 72.07
Precision: 71.02, Recall: 75.48, F1-Score: 73.18
Precision: 74.02, Recall: 68.28, F1-Score: 71.04
Precision: 72.29, Recall: 72.32, F1-Score: 72.31
Precision: 72.96, Recall: 71.88, F1-Score: 72.42
Precision: 72.24, Recall: 73.92, F1-Score: 73.07
Precision: 73.28, Recall: 71.84, F1-Score: 72.55
Precision: 75.82, Recall: 64.84, F1-Score: 69.90
Precision: 71.12, Re

Precision: 74.23, Recall: 69.72, F1-Score: 71.91
Precision: 71.36, Recall: 75.36, F1-Score: 73.31
Precision: 73.96, Recall: 70.20, F1-Score: 72.03
Precision: 73.66, Recall: 71.92, F1-Score: 72.78
Precision: 74.99, Recall: 69.56, F1-Score: 72.17
Precision: 74.50, Recall: 69.20, F1-Score: 71.75
Precision: 74.33, Recall: 69.48, F1-Score: 71.82
Precision: 73.42, Recall: 71.60, F1-Score: 72.50
Precision: 73.38, Recall: 72.12, F1-Score: 72.75
Precision: 74.18, Recall: 70.80, F1-Score: 72.45
Precision: 75.11, Recall: 68.08, F1-Score: 71.42
Precision: 73.00, Recall: 73.12, F1-Score: 73.06
Precision: 75.41, Recall: 66.60, F1-Score: 70.73
Precision: 71.71, Recall: 73.80, F1-Score: 72.74
Precision: 73.08, Recall: 70.04, F1-Score: 71.53
Precision: 75.77, Recall: 63.56, F1-Score: 69.13
Precision: 71.30, Recall: 74.84, F1-Score: 73.03
Precision: 73.09, Recall: 70.72, F1-Score: 71.88
Precision: 71.64, Recall: 73.24, F1-Score: 72.43
Precision: 72.89, Recall: 70.88, F1-Score: 71.87
Precision: 72.36, Re

Precision: 75.56, Recall: 66.04, F1-Score: 70.48
Precision: 72.47, Recall: 73.92, F1-Score: 73.19
Precision: 74.49, Recall: 69.72, F1-Score: 72.02
Precision: 72.99, Recall: 71.88, F1-Score: 72.43
Precision: 74.42, Recall: 67.96, F1-Score: 71.04
Precision: 72.65, Recall: 71.08, F1-Score: 71.86
Precision: 74.75, Recall: 67.84, F1-Score: 71.13
Precision: 72.51, Recall: 72.68, F1-Score: 72.59
Precision: 73.52, Recall: 70.96, F1-Score: 72.22
Precision: 73.35, Recall: 71.12, F1-Score: 72.22
Precision: 74.35, Recall: 67.84, F1-Score: 70.95
Precision: 73.77, Recall: 70.08, F1-Score: 71.88
Precision: 74.38, Recall: 67.48, F1-Score: 70.76
Precision: 72.64, Recall: 72.52, F1-Score: 72.58
Precision: 73.28, Recall: 70.52, F1-Score: 71.87
Precision: 74.41, Recall: 69.44, F1-Score: 71.84
Precision: 73.46, Recall: 70.52, F1-Score: 71.96
Precision: 72.20, Recall: 71.68, F1-Score: 71.94
Precision: 75.49, Recall: 62.96, F1-Score: 68.66
Precision: 69.33, Recall: 77.48, F1-Score: 73.18
Precision: 75.44, Re

Precision: 76.12, Recall: 66.04, F1-Score: 70.72
Precision: 71.99, Recall: 73.72, F1-Score: 72.85
Precision: 73.82, Recall: 68.48, F1-Score: 71.05
Precision: 71.37, Recall: 73.60, F1-Score: 72.47
Precision: 74.37, Recall: 66.40, F1-Score: 70.16
Precision: 71.42, Recall: 73.28, F1-Score: 72.34
Precision: 74.21, Recall: 67.32, F1-Score: 70.60
Precision: 74.01, Recall: 68.36, F1-Score: 71.08
Precision: 72.56, Recall: 70.64, F1-Score: 71.58
Precision: 73.96, Recall: 67.84, F1-Score: 70.77
Precision: 72.95, Recall: 70.44, F1-Score: 71.67
Precision: 73.86, Recall: 68.04, F1-Score: 70.83
Precision: 71.42, Recall: 73.56, F1-Score: 72.47
Precision: 72.75, Recall: 69.72, F1-Score: 71.20
Precision: 71.82, Recall: 71.76, F1-Score: 71.79
Precision: 73.19, Recall: 68.92, F1-Score: 70.99
Precision: 72.79, Recall: 70.72, F1-Score: 71.74
Precision: 72.94, Recall: 70.52, F1-Score: 71.71
Precision: 74.25, Recall: 68.64, F1-Score: 71.34
Precision: 71.68, Recall: 73.52, F1-Score: 72.59
Precision: 73.42, Re